In [1]:
import pandas as pd
from scipy.stats import ttest_ind
import torch
from google.colab import files

In [2]:
uploaded = files.upload()  # This will prompt you to upload a file

Saving Stimulus_FYP.csv to Stimulus_FYP.csv


In [3]:
file= "Stimulus_FYP.csv"

df = pd.read_csv(file, sep=";")

print(df.columns.tolist())
#df.head()

['sr_no', 'headlines', 'headline_no', 'negative', 'neutral', 'positive', 'chatgpt']


In [4]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

# Preprocess text (username and link placeholders) #code obtained from the authors, did not make much changes here since this is mostly irrelevant and specific to tweets...
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

# PT MODEL (IMP)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

model.save_pretrained(MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

In [5]:
#creating a processing function
def process_text(text):
    text = preprocess(text)  # Apply your preprocessing function
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    # Define the fixed order
    label_order = ["negative", "neutral", "positive"]

    # Map labels to their respective indices
    id2label = {v: k for k, v in config.id2label.items()}

    # Retrieve scores based on the fixed order
    results = [(label, round(float(scores[id2label[label]]), 4)) for label in label_order]

    return results  # Returns a list of (label, score) tuples

In [6]:
# Apply the function to each row in the 'Emotional' column
df["Scores"] = df["headlines"].apply(process_text)

# Extract values into separate columns
df["Negative"] = df["Scores"].apply(lambda x: x[0][1])
df["Neutral"] = df["Scores"].apply(lambda x: x[1][1])
df["Positive"] = df["Scores"].apply(lambda x: x[2][1])

In [7]:
# Drop the original column if not needed
df.drop(columns=["Scores"], inplace=True)

# Display the first few processed rows
df.head()

,sr_no,headlines,headline_no,negative,neutral,positive,chatgpt,Negative,Neutral,Positive
0,1,Stephen Hawking says A.I. could be ‘worst even...,T1E,0.8509,0.1374,0.0117,6,0.8509,0.1374,0.0117
1,2,"As vaccinations lag, Italy's elderly again pay...",T2E,0.7366,0.2465,0.0168,5,0.7366,0.2465,0.0168
2,3,Bent the knee: Outrage as NASA scrubs women an...,T3E,0.7722,0.2196,0.0082,6,0.7722,0.2196,0.0082
3,4,Scheming' AI bot ChatGPT tried to stop itself ...,T4E,0.8201,0.1714,0.0086,6,0.8201,0.1714,0.0086
4,5,Google's Gemini turns villain: AI asks user to...,T5E,0.8648,0.1286,0.0066,7,0.8769,0.1168,0.0062


In [ ]:
df.to_csv('Stimuli_new.csv', index=False)
files.download('Stimuli_new.csv') #download the csv, that is then uploaded in R to conduct the pre-test analysis (t-test for negative and neutral scores)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>